In [171]:
## Write list to excel


# import jsonlines


# lst = []
# with jsonlines.open('../docs/train_input.jsonl', 'r') as jsonl_f:
#     for item in jsonl_f:
#         lst.append(item)

# import openpyxl


# excel_list = [(item['messages'][0]['content'], item['messages'][1]['content'] )for item in lst]
# wb = openpyxl.Workbook()
# sheet = wb.active
# sheet.cell(row=1,column=1).value= 'Вопрос'
# sheet.cell(row=1,column=2).value = 'Ответ'
# for enum, values in enumerate(excel_list):
#     sheet.cell(row=enum+2,column=1).value = values[0]
#     sheet.cell(row=enum+2,column=2).value = values[1]
# wb.save('../docs/scripts_input.xlsx')

### Считать экзель файл со скриптами

In [172]:
import openpyxl

excel_script_path = '../docs/scripts_input.xlsx'
wb = openpyxl.load_workbook(excel_script_path)
ws = wb.active
scripts= []
for row in ws.iter_rows(min_row=2):
    scripts.append((row[0].value, row[1].value))
scripts[0]

('Привет!',
 'Добрый день! 👋 Рады видеть вас в {company_name}. Как мы можем помочь вам сегодня?')

### Считать информацию по компании

In [173]:
import json

# Opening JSON file
info_file = open('../docs/info.json')
info = json.load(info_file)

In [174]:
company_name = info['company_name']
phone_number = info['phone_number']
site_url = info['site_url']
courses = info['courses']

### Считать пропуски в скриптах информацией о компании

In [175]:
scripts = [(q, a.format(company_name=company_name, phone_number=phone_number, site_url=site_url, courses=courses)) for q, a in scripts]
scripts[0]

('Привет!',
 'Добрый день! 👋 Рады видеть вас в Университет НЛП. Как мы можем помочь вам сегодня?')

### Считать системный промпт

In [176]:
system_prompt = ''
with open('../docs/system_prompt.txt', 'r') as f:
    system_prompt = f.read()
system_prompt

'Ты чат-бот для продаж курсов по нейропсихологии от компании {company_name}. Ты должен отвечать на вопросы лидов в телеграм-боте. Приветствовать, давать информацию по курсам, отправлять контакты для связи и ссылки на сайт и другие социальные сети.'

### Заполнить пропуски в системном промпте информацией о компании

In [177]:
system_prompt = system_prompt.format(company_name=company_name, phone_number=phone_number, site_url=site_url, courses=courses)
system_prompt

'Ты чат-бот для продаж курсов по нейропсихологии от компании Университет НЛП. Ты должен отвечать на вопросы лидов в телеграм-боте. Приветствовать, давать информацию по курсам, отправлять контакты для связи и ссылки на сайт и другие социальные сети.'

### Сохранить заполненные скрипты в JSONL файле обучения

In [178]:
jsonl_data = []
jsonl_data.extend([{"messages":[{"role":"system","content":system_prompt}, {"role":"user","content": a}, {"role":"assistant","content": q}]} for a, q in scripts])

In [179]:
import json

def dump_jsonl(data, output_path, append=False):
    """
    Write list of objects to a JSON lines file.
    """
    mode = 'a+' if append else 'w'
    with open(output_path, mode, encoding='utf-8') as f:
        for line in data:
            json_record = json.dumps(line, ensure_ascii=False)
            f.write(json_record + '\n')

In [180]:
json_script_path = '../docs/train_output.jsonl'
dump_jsonl(jsonl_data, json_script_path)

In [183]:
scripts_text = ''.join(['Вопрос: ' + q + '\nОтвет: ' + a + '\n' for q, a in scripts])
scripts_text

"Вопрос: Привет!\nОтвет: Добрый день! 👋 Рады видеть вас в Университет НЛП. Как мы можем помочь вам сегодня?\nВопрос: Мне интересно узнать больше о курсах по нейропсихологии.\nОтвет: У нас есть различные курсы по нейропсихологии, которые охватывают от начальных до продвинутых уровней. Вы можете начать с 'Основ нейропсихологии' или погрузиться в специализированные темы, такие как 'Нейропсихология обучения' и 'Эмоциональный интеллект'. Что вас больше всего интересует?\nВопрос: Как мне записаться на курс?\nОтвет: Для записи на курс вам нужно перейти на наш сайт https://neuro-moscow-nlp.ru и выбрать интересующий вас курс. Там вы найдете форму регистрации и инструкции по оплате. Если у вас возникнут вопросы или вам понадобится помощь, вы всегда можете связаться с нами через контакт-форму на сайте или по телефону +123456789.\nВопрос: Сколько стоит курс?\nОтвет: Цены на курсы варьируются в зависимости от их длительности и уровня сложности. Например, Основы нейропсихологии стоит 700р, а курс Не

In [184]:
with open('../docs/train_script.txt', 'w') as f:
    f.write(scripts_text)